In [ ]:
import numpy, cv2, time
from matplotlib import pyplot as plt
from matplotlib import cm
from JokeFunc import GrayImg
from importlib import reload
GrayImg = reload(GrayImg)

In [ ]:
img_shape = (128,128)
center = (img_shape[0]//2,img_shape[1]//2)
circle_ref = cv2.circle(numpy.zeros(img_shape),center,img_shape[0]//10,255,-1)
circle_temp = cv2.circle(numpy.zeros(img_shape),center,img_shape[0]*3//20,255,-1)
ellipse_temp = cv2.ellipse(numpy.zeros(img_shape),center,(img_shape[0]*3//20,img_shape[0]//10),0,0,360,255,-1)
figure = plt.figure(figsize=(15,15))
figure.add_subplot(1,3,1)
plt.title('Reference Image')
plt.imshow(circle_ref,cmap='gray')
figure.add_subplot(1,3,2)
plt.title('Template Image 1')
plt.imshow(circle_temp,cmap='gray')
figure.add_subplot(1,3,3)
plt.title('Template Image 2')
plt.imshow(ellipse_temp,cmap='gray')
plt.show()

In [ ]:
hand_ref = cv2.imread(r'C:\Users\Admin\ImageRegistration\Image\R1.png',32)
hand_temp = cv2.imread(r'C:\Users\Admin\ImageRegistration\Image\T1.png',32)
hand_ref = hand_ref[:-1,:] 
hand_temp = hand_temp[:-1,:]
figure = plt.figure(figsize=(10,10))
figure.add_subplot(1,2,1)
plt.title('Reference Image')
plt.imshow(hand_ref,cmap='gray',vmax=255,vmin=0)
figure.add_subplot(1,2,2)
plt.title('Template Image')
plt.imshow(hand_temp,cmap='gray',vmax=255,vmin=0)
plt.show()
hand_ref.shape,hand_temp.shape

# Explicit Time Marching

In [ ]:
#ref_img = numpy.float32(circle_ref)/255
ref_img = numpy.float32(hand_ref)/255
#temp_img = numpy.float32(circle_temp)/255
#temp_img = numpy.float32(ellipse_temp)/255
temp_img = numpy.float32(hand_temp)/255

m,n = ref_img.shape
X = numpy.arange(0,m,1)
Y = numpy.arange(0,n,1)
X, Y = numpy.meshgrid(X, Y)

t1 = time.time()

Imax=1000
alpha=0.02
tau=2
res_eps=5e-2
diff_eps=1e-5

m, n = ref_img.shape
u = numpy.zeros((2,m,n))
res0_exist = False

for k in range(Imax):
    u_old = numpy.copy(u)
    
    #extended u by neuman BCs
    ex_u = numpy.zeros((2,m+2,n+2))
    ex_u[:,1:-1,1:-1] = u
    ex_u[:,0,1:-1] = u[:,0,:]
    ex_u[:,1:-1,0] = u[:,:,0]
    ex_u[:,-1,1:-1] = u[:,-1,:]
    ex_u[:,1:-1,-1] = u[:,:,-1]
    
    #laplacian of u
    Lp_u = ex_u[:,:-2,1:-1]+ex_u[:,1:-1,:-2]+ex_u[:,2:,1:-1]+ex_u[:,1:-1,2:]-4*u
    ex_Tu = GrayImg.warp_vec(temp_img,ex_u,up=1,left=1,right=1,down=1)
    Tu = ex_Tu[1:-1,1:-1]
    Tu_dx = (ex_Tu[2:,1:-1]-ex_Tu[:-2,1:-1])/2 #central
    Tu_dy = (ex_Tu[1:-1,2:]-ex_Tu[1:-1,:-2])/2 #central
    Tu_du = numpy.zeros((2,m,n))
    Tu_du[0,:,:] = Tu_dx
    Tu_du[1,:,:] = Tu_dy
    
    Res = alpha*Lp_u-(Tu-ref_img)*Tu_du
    norm_Res = numpy.linalg.norm(Res)
    if not res0_exist:
        res0 = norm_Res
        res0_exist = True
        
    rel_res = norm_Res/res0
    
    regist_Diff_ETM_img = numpy.clip(Tu,0,255)
    Diff_regImg_refImg_img = abs(regist_Diff_ETM_img-ref_img)
    
    print_every = 50
    if k%print_every==0:
        t3 = time.time()
        print('rel_res =',rel_res)
        
        #plt.imshow(Diff_regImg_refImg_img,cmap='gray')
        #plt.show()
        fig = plt.figure(figsize=(15,15))
        fig.add_subplot(1,3,1)
        plt.imshow(Diff_regImg_refImg_img,cmap='gray')
        ux = fig.add_subplot(132,projection='3d')
        plt.title('ux')
        ux.plot_surface(X, Y, u[0,:,:], cmap=cm.coolwarm,linewidth=0, antialiased=False)
        uy = fig.add_subplot(133,projection='3d')
        plt.title('uy')
        uy.plot_surface(X, Y, u[1,:,:], cmap=cm.coolwarm,linewidth=0, antialiased=False)
        plt.show()
        
    if rel_res<res_eps:
        print('solution converges')
        break

    #update u
    u = u+tau*F
    
    diff = numpy.linalg.norm(u-u_old)
    if (k+1)%print_every==0:
        print('time =',time.time()-t3)
        print('iteration =',k+1,'\ndiff = ',diff)
    if diff<diff_eps:
        print('solution dose not change')
        break
        
if k==Imax-1:
    print('Exceed')
t2 = time.time()
print('\n\niteration =',k+1,'\ndiff = ',diff,'rel_res =',rel_res,'total_time =',t2-t1)
fig = plt.figure(figsize=(15,15))
fig.add_subplot(1,3,1)
plt.title('Reference Image')
plt.imshow(ref_img,cmap='gray')
fig.add_subplot(1,3,2)
plt.title('Registed Image')
plt.imshow(regist_Diff_ETM_img,cmap='gray')
fig.add_subplot(1,3,3)
plt.title('Template Image')
plt.imshow(temp_img,cmap='gray')
plt.show()

